In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tournamentgym import TournamentEnv
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import traceback
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
import hyperopt 
from pprint import pprint
import graphviz


c:\Users\standard\OneDrive\Documents\Git\Tournament-gym\tournamentgym.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import stable_baselines3 as sb3
from stable_baselines3 import PPO,A2C,DQN
from sb3_contrib import RecurrentPPO,ARS,QRDQN
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.env_util import make_vec_env
import torch
from torch.utils.data import DataLoader, Dataset


In [3]:
torch.cuda.is_available()

True

In [4]:
import tensorboard
## tensor board_log
tensorboard_log = "Logs/"

In [5]:

team_data=pd.read_csv('Process_data/M_pm_w_names.csv')
env=TournamentEnv(season=2022,team_stats=team_data,loading_bar=False)


## Train the Models ->2019

In [6]:
##hyper param Search
space=hp.choice('model_type',
    [

        {'model':RecurrentPPO,
         'model_kwargs':{
                    'policy':'MlpLstmPolicy',
                    'learning_rate':hp.uniform('PPO_learning_rate',1e-7,0.1),
                    'n_steps':hp.choice('PPO_n_steps',[c for c in range(10,200,2)]),
                    'gamma':hp.uniform('PPO_gamma',1e-4,1),
                    'gae_lambda':hp.uniform('PPO_gae_lambda',1e-4,1),
                    'batch_size':hp.choice('PPO_batch_size',[c for c in range(2,200,2)]),
                    'n_epochs':hp.choice('PPO_n_epochs',[c for c in range(10,20,2)]),
                    'policy_kwargs':{
                                    'net_arch':list([hp.choice('PPO_net_arch',[64,128,256])])
                                    },

                                    # 'lr_schedule':hp.lognormal('PPO_lr_schedule',1e-6,.01),
                                    },
                },
        
        # {'model':QRDQN,
        #  'model_kwargs':{
        #             'policy':'MlpPolicy',

        #             'learning_rate':hp.uniform('DQN_learning_rate',1e-7,0.1),
        #             'train_freq':hp.choice('DQN_train_freq',[c for c in range(2,68,2)]),

        #             'buffer_size':hp.choice('DQN_buffer_size',[c for c in range(2,68,2)]),
        #             'batch_size':hp.choice('DQN_batch_size',[c for c in range(2,200,2)]),
        #             'tau':hp.uniform('DQN_tau',1e-4,1),
        #             'gamma':hp.uniform('DQN_gamma',1e-4,1),

        #             'exploration_fraction':hp.uniform('DQN_exploration_fraction',1e-4,1),
        #             'exploration_initial_eps':hp.uniform('DQN_exploration_initial_eps',1e-4,1),
        #             'exploration_final_eps':hp.uniform('DQN_exploration_final_eps',1e-7,1e-4),

        #             'target_update_interval':hp.choice('DQN_target_update_interval',[c for c  in range (2,100,2)]),
        #             'policy_kwargs':{
        #                             'net_arch':list([hp.choice('DQN_net_arch',[64,128,256])]),
        #                             # 'lr_schedule':hp.lognormal('DQN_lr_schedule',1e-6,.01),
        #                             },
        #         },
        #     },

        {'model':ARS,
         'model_kwargs':{
                    'policy':hp.choice('ARS_policy',['MlpPolicy','LinearPolicy']),
                    'learning_rate':hp.uniform('ARS_learning_rate',1e-7,0.1),
                    'n_delta':hp.choice('ARS_n_delta',[c for c in range(2,68,2)]),

                    # 'zero_policy':hp.choice('ARS_zero_policy',[True,False]),
                    'policy_kwargs':{
                                    'net_arch':list([hp.choice('ARS_net_arch',[64,128,256])])
                                    },
                                    # 'lr_schedule':hp.lognormal('ARS_lr_schedule',1e-6,.01),
                        },
        },
    ]
)
sample_space=hyperopt.pyll.stochastic.sample(space)
sample_space

{'model': sb3_contrib.ppo_recurrent.ppo_recurrent.RecurrentPPO,
 'model_kwargs': {'batch_size': 68,
  'gae_lambda': 0.7014687552624854,
  'gamma': 0.20752031241878421,
  'learning_rate': 0.07635657807640692,
  'n_epochs': 18,
  'n_steps': 26,
  'policy': 'MlpLstmPolicy',
  'policy_kwargs': {'net_arch': (256,)}}}

In [7]:
train_options=dict(season=2022,team_stats=team_data,discrete=False,loading_bar=False)
test_options=dict(season=2023,team_stats=team_data,discrete=False,loading_bar=False,exclude_seasons=[c for c in range(2003,2023)])



        

In [8]:
def objective(space):
    try:
        num_envs=10

        test_multi_env=sb3.common.env_util.make_vec_env(TournamentEnv,n_envs=num_envs,env_kwargs=test_options)
        train_multi_env=sb3.common.env_util.make_vec_env(TournamentEnv,n_envs=num_envs,env_kwargs=train_options)

        model=space['model']
        model_args=space['model_kwargs']
        policy=model_args['policy']
        if policy=='LinearPolicy':
            model_args.pop('policy_kwargs')
            
        model = model(env=train_multi_env,tensorboard_log='Logs/',device='cuda', verbose=0,**model_args)

        # stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=100, min_evals=10, verbose=0)
        # eval_callback = EvalCallback(model.env, eval_freq=2,  verbose=0)

        model.learn(50000)

        model.set_env(test_multi_env)
        
        obs=model.env.reset()
        states=None
        episode_starts = np.ones((num_envs,), dtype=bool)
        done=[False]
        while not done[0]:
            acts,states=model.predict(observation=obs,state=states,episode_start=episode_starts,deterministic=True)
            obs, reward, done, info = model.env.step(acts)
            episode_starts = done

        rew=info[0]['score']
        return {'loss':float(-rew),'status':STATUS_OK}


    except Exception as e:
        pprint(space)
        traceback.print_exc(e)



In [9]:

trails=Trials()
best=fmin(fn=objective,
         space=space,
         algo=tpe.suggest,
         max_evals=200,
         trials=trails)
best

  4%|▍         | 9/200 [55:13<28:10:07, 530.93s/trial, best loss: -47.0]

In [ ]:
import pickle
pickle.dump(trails, open("Day2_trials.pkl", "wb"))
trials = pickle.load(open("my_trials.pkl", "rb"))


In [ ]:
env_options=dict(season=2022,team_stats=team_data,loading_bar=False)
# best=hyperopt.pyll.stochastic.sample(space)
pprint(best)
best_model_type=best['model']
model_args=best['model_kwargs']
policy=model_args['policy']

if policy=='LinearPolicy':

    model_args.pop('policy_kwargs')



In [ ]:

train_env=sb3.common.env_util.make_vec_env(TournamentEnv,n_envs=1,env_kwargs=train_options)
test_env=sb3.common.env_util.make_vec_env(TournamentEnv,n_envs=1,env_kwargs=test_options)
best_model = best_model_type( env=train_env,
                                tensorboard_log='Logs/',
                                device='cuda',
                                **model_args)
# stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=100, min_evals=10, verbose=0)
eval_callback = EvalCallback(best_model.env, eval_freq=50,  verbose=0)

best_model.learn(50000,callback=eval_callback)
# best_model.learn(1)

In [ ]:
best_model.set_env(test_env)
obs=best_model.env.reset()
states=None
episode_starts = np.ones((1,), dtype=bool)
done=[False]
while not done[0]:

    acts,states=best_model.predict(observation=obs,state=states,episode_start=episode_starts,deterministic=True)
    obs, reward, done, info = best_model.env.step(acts)
    episode_starts=done

In [ ]:
info[0]

In [ ]:
best_dqn={'batch_size': 37,
 'buffer_size': 2,
 'exploration_final_eps': 3.450102881484894e-05,
 'exploration_fraction': 0.9148110500687865,
 'exploration_initial_eps': 0.09837129661293152,
 'gamma': 0.2731720156175311,
 'learning_rate': 0.025123870821062953,
 'net_arch': 0,
 'target_update_interval': 15,
 'tau': 0.8416411197389558,
 'train_freq': 6,
}

In [ ]:
test_model.set_env(multi_env)

env=test_model.env
obs=env.reset()
done=[False]
states=None
while not any(done):
    acts,states=test_model.predict(obs,state=states,episode_start=done,deterministic=True)

    obs,rew,done,info=env.step(acts)


In [ ]:
info[1]['score']

In [ ]:
## test one 2022
obs = test_env.reset(season=2023,weight_scores=False,track_wins=True)
done = False
while not done:
    action = model.predict(obs,deterministic =True)[0]
    obs, reward, done, info = test_env.step(action)
info

In [ ]:
## test individually
scores=[]
max_scores=[]
model_scores={}
dfs={}
for i,model in enumerate(models):

    obs = env.reset(season=2022,weight_scores=False,track_wins=False)
    done = False
    while not done:
        action = model.predict(obs,deterministic=False)[0]

        obs, reward, done, info = env.step(action)
    total_score=info['total_score']
    num_correct=info['num_correct']
    scores.append(total_score) 
    dfs[total_score]=env.action_data
    model_scores[total_score]=model
    name=model.__repr__().split(' ')[0].split('.')[-1]
    env.render(score=f'{name} {i} Correct:{num_correct} Score:{total_score}')
display(info['total_score'],info['num_correct'])    


In [ ]:
## pick out the best Models
scores=sorted(scores)
topscore=scores[-1]
secondscore=scores[-3]
print(scores)
top_model=model_scores[topscore]
second_model=model_scores[secondscore]


## predict This year

In [ ]:

obs = env.reset(season=2023,weight_scores=False,track_wins=True)
done = False
while not done:
    actions = [model.predict(obs,deterministic =False)[0] for model in models]
    action=np.mean(actions, axis=0)

    obs, reward, done, info = env.step(action)

env.render(score=f"Ensembled ARS Prediction")

In [ ]:

obs = env.reset(season=2023,weight_scores=False,track_wins=False)
done = False
while not done:
    action = top_model.predict(obs,deterministic=False)[0]
    obs, reward, done, info = env.step(action)
name=top_model.__repr__().split(' ')[0].split('.')[-1]
env.render(score=f'Top {name}  Pred')

In [ ]:
obs = env.reset(season=2023,weight_scores=False,track_wins=False)
done = False
while not done:
    action = second_model.predict(obs,deterministic=False)[0]
    obs, reward, done, info = env.step(action)
name=second_model.__repr__().split(' ')[0].split('.')[-1]
env.render(score=f'Second Best {name} Pred')

In [ ]:
# for i,model in enumerate(models):

#     obs = env.reset(season=2023,weight_scores=False,track_wins=False)
#     done = False
#     while not done:
#         action = model.predict(obs,deterministic=True)[0]

#         obs, reward, done, info = env.step(action)
#     env.render(score=f'Model {i} Pred')


jgfulfgu